In [3]:
# debug cell
with open('simple_table.txt', 'r') as f:
    print(type(f))
    cnt = 2
    for i,line in enumerate(f):
        print(i, ':\t', line)
        cnt -= 1
        if cnt == 0:
            break

<class '_io.TextIOWrapper'>
0 :	 CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO:AF	UKB-b-6134:ES	UKB-b-6134:SE	UKB-b-6134:LP	UKB-b-6134:AF	UKB-b-6134:SS	UKB-b-6134:EZ	UKB-b-6134:SI	UKB-b-6134:NC	UKB-b-6134:ID

1 :	 1	49298	rs10399793	T	C	.	PASS	0.623238	0.0017795	0.00366743	0.200659	0.623238	.	.	.	.	rs10399793



In [4]:
# debug cell
import gzip
with gzip.open('./ukb-b-6134.vcf.gz') as f:
    print(f)

<gzip _io.BufferedReader name='./ukb-b-6134.vcf.gz' 0x7f1d5852f810>


In [5]:
import os
import json
import pickle
import time
import gzip
import shutil

def generate_gwas_output_as_tsv_file(inVCF_File, writeFile=None):
    root = './tabular_format_gwas_data/'
    t0 = time.time()
    if writeFile is None:
        writeFile = root + inVCF_File.split('/')[-1].split('.')[0] + '.tsv'
    
    print("Processing:", inVCF_File)
    if ".gz" == inVCF_File[-3:]:
        with gzip.open(inVCF_File, 'rb') as f_in:
            with open(root+'temp_vcf.vcf', 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        inVCF_File = root+'temp_vcf.vcf'
    """
    Convert all the VCF into simplified tsv format
    Source: https://github.com/everestial/VCF-simplify#table-of-contents
    python3 ./VCF-Simplify/VCF-Simplify/VcfSimplify.py SimplifyVCF -toType table -inVCF ./ukb-b-6134.vcf -outFile ./simple_table.txt
    """
    vcf_simplify_path = "./VCF-Simplify/VCF-Simplify/VcfSimplify.py"
    out_File = root+'temp_table.tsv'
    os.system(f"python3 {vcf_simplify_path} SimplifyVCF -toType table -inVCF {inVCF_File} -outFile {out_File}")
        
    f_read = open(out_File, 'r')
    f_write = open(writeFile, 'w')
    cnt = 0
    for line in f_read:
    #     print(line)
        newliner = ''
        if '\n' == line[-1]:
            newliner = '\n'
            line = line.strip()
        if 'CHROM' in line:
            line = line.upper() + '\tPVAL_generated_from_LP'
        else:
            LP = float(line.split('\t')[10])
            p_val = str(10**(-1 * LP))
            line = line + '\t' + p_val
        line = line + newliner
    #     print(line)
        f_write.write(line)
        cnt += 1
    #     if cnt == 20: break

    f_read.close()
    f_write.close()
    print(f'Total SNPs: {cnt} | Total exec time: {time.time() - t0} seconds')

In [6]:
# generate_gwas_output_as_tsv_file(inVCF_File='./ukb-b-6134.vcf.gz', writeFile=None)

In [7]:

# ' abc def\n'.strip()

In [8]:
# 10**(-1*0.200659) #0.6300007

In [9]:
# %pip install openpyxl
import pandas as pd

category_map = {
    'Continuous' : 'beta', 
    'Categorical Ordered (assumed continuous)': 'beta',
    'Binary': 'or',
    'NA (Possibly binary)': 'or'
}

df = pd.read_excel('GWAS_datasets_to_consider.xlsx', engine='openpyxl')[['GWAS_ID', 'Trait', 'Category']]
print(df['Category'].unique())
GWAS_ID = 'ieu-b-111'
category = category_map[df[df['GWAS_ID'] == GWAS_ID].to_numpy()[0, 2]] 
category

['Continuous' 'Binary' 'Risk factor' 'NA (Possibly binary)'
 'Categorical Ordered (assumed continuous)']


'beta'

In [10]:
from glob import glob
# all_gwas_vcf_files = glob('/mnt/c/Users/HP/Downloads/Compressed/GWAS/*')
ROOT = '/mnt/c/Users/HP/Downloads/Compressed/GWAS/'
ALL_GWAS_IDS = list(set(df['GWAS_ID'].to_numpy().tolist()) - {'bbj-a-46', 'bbj-a-78', 'ieu-a-1283', 'ukb-a-257'})
base_files = {}
for GWAS_ID in ALL_GWAS_IDS:
    print(GWAS_ID)
    file = ROOT + GWAS_ID + '.vcf.gz'
    base_files[GWAS_ID] = category_map[df[df['GWAS_ID'] == GWAS_ID].to_numpy()[0, 2]] 
#     generate_gwas_output_as_tsv_file(inVCF_File=file, writeFile=None)

ukb-b-12064
ukb-b-7663
ukb-b-18275
ukb-b-19953
ukb-d-20405_0
ukb-d-20405_1
ieu-b-25
ukb-b-13806
ukb-b-4424
ieu-b-110
ukb-d-20405_2
ieu-b-39
ukb-b-3957
ukb-b-6134
ukb-b-6324
ukb-b-8476
ieu-b-38
met-d-Total_C
ieu-b-109
ieu-b-111
ukb-b-17627
ukb-b-2209
ukb-b-770


In [11]:
base_files

{'ukb-b-12064': 'or',
 'ukb-b-7663': 'or',
 'ukb-b-18275': 'or',
 'ukb-b-19953': 'beta',
 'ukb-d-20405_0': 'or',
 'ukb-d-20405_1': 'or',
 'ieu-b-25': 'beta',
 'ukb-b-13806': 'or',
 'ukb-b-4424': 'beta',
 'ieu-b-110': 'beta',
 'ukb-d-20405_2': 'or',
 'ieu-b-39': 'beta',
 'ukb-b-3957': 'beta',
 'ukb-b-6134': 'beta',
 'ukb-b-6324': 'beta',
 'ukb-b-8476': 'or',
 'ieu-b-38': 'beta',
 'met-d-Total_C': 'beta',
 'ieu-b-109': 'beta',
 'ieu-b-111': 'beta',
 'ukb-b-17627': 'beta',
 'ukb-b-2209': 'beta',
 'ukb-b-770': 'beta'}

In [12]:
gender_map = {'Female': 0,'Male': 1}
ADNIMERGE = pd.read_csv('ADNIMERGE.csv')
def get_gender_and_age(PTID):
    gender = ADNIMERGE[ADNIMERGE['PTID'] == PTID]['PTGENDER'].dropna().unique()[0]
    age = ADNIMERGE[ADNIMERGE['PTID'] == PTID]['AGE'].dropna().unique()[0]
    return str(gender_map[gender]) + ' ' + str(age)

NUM_TRAINING_SAMPLES = int(830 * .7)
print('NUM_TRAINING_SAMPLES:', NUM_TRAINING_SAMPLES)
f_writable = open('./COVAR_FILE.txt', 'w')
f_TRAINING_SAMPLES = open('./TRAINING_SAMPLES.txt', 'w')
with open('/mnt/c/Users/HP/Documents/GWA_tutorial/2_Population_stratification/covar_mds.txt') as f:
    for i, line in enumerate(f):
        if 'FID IID' in line:
            line = line[:-1] + ' GENDER AGE\n'
        else:
            line = line[:-1] + ' ' + get_gender_and_age(PTID=line.split(' ')[1]) + '\n'
        if i < NUM_TRAINING_SAMPLES:
            f_TRAINING_SAMPLES.write(' '.join(line.split(' ')[:2])+'\n')
        f_writable.write(line)
#         print(line)
f_writable.close()

/home/gene/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (18,19,20,103,104) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


NUM_TRAINING_SAMPLES: 581


In [15]:
def run_prsice(GWAS_ID, redo=False):
    '''
    return_status --> 0: success | 256: Error. Execusion Halted | 2: 
    '''
    return_status = os.system(f'mkdir ./prsice_output/{GWAS_ID}/')
    print('0:', return_status)
    if return_status == 256 and not redo:
        return return_status
    prsice_command = f'Rscript ./PRSice_linux/PRSice.R \
        --dir ./prsice_output \
        --prsice ./PRSice_linux/PRSice_linux \
        --base ./tabular_format_gwas_data/{GWAS_ID}.tsv \
        --target /mnt/c/Users/HP/Documents/GWA_tutorial/2_Population_stratification/ADNI \
        --thread 64 \
        --stat {base_files[GWAS_ID].upper()} \
        --{base_files[GWAS_ID]} \
        --binary-target F \
        --quantile 10 \
        --out ./prsice_output/{GWAS_ID}/{GWAS_ID} \
        --snp ID --chr CHROM --bp POS --A2 REF --A1 ALT --stat {GWAS_ID.upper()}:ES --pvalue PVAL_generated_from_LP \
        --score std \
        --cov ./COVAR_FILE.txt'
    print(prsice_command)
#     return_status = os.system(prsice_command + '  --keep ./TRAINING_SAMPLES.txt') # generate the file named {GWAS_ID}.valid
    return_status = os.system(prsice_command) # generate the file named {GWAS_ID}.valid
    print('1a:', return_status)
    if os.path.isfile(f'./prsice_output/{GWAS_ID}/{GWAS_ID}.valid'): 
        prsice_command += f'  --extract ./prsice_output/{GWAS_ID}/{GWAS_ID}.valid' 
#         return_status = os.system(prsice_command + '  --keep ./TRAINING_SAMPLES.txt') # perform the regression analysis on the training samples only and choose the best p-value-threshold
        return_status = os.system(prsice_command) # perform the regression analysis on the training samples only and choose the best p-value-threshold
        print('1b:', return_status)
#     return_status = os.system(prsice_command + '  --all-score --no-regress') # Do not perform the regression analysis and simply output all PRS.
#     return_status = os.system(prsice_command + '  --all-score') # Do not perform the regression analysis and simply output all PRS.
#     print('2:', return_status)
    # in later steps, select the PRS values for the selected p-value-threshold for all the samples (both training and testing)
    return_status = os.system('echo "===== Done ====="')
    print('2:', return_status) 
#     os.system(f'mv ./prsice_output/{GWAS_ID}.* ./prsice_output/{GWAS_ID}/')

In [16]:
for GWAS_ID in base_files:
    print(GWAS_ID)
    run_prsice(GWAS_ID=GWAS_ID)
    break

ukb-b-12064
0: 0
Rscript ./PRSice_linux/PRSice.R         --dir ./prsice_output         --prsice ./PRSice_linux/PRSice_linux         --base ./tabular_format_gwas_data/ukb-b-12064.tsv         --target /mnt/c/Users/HP/Documents/GWA_tutorial/2_Population_stratification/ADNI         --thread 64         --stat OR         --or         --binary-target F         --quantile 10         --out ./prsice_output/ukb-b-12064/ukb-b-12064         --snp ID --chr CHROM --bp POS --A2 REF --A1 ALT --stat UKB-B-12064:ES --pvalue PVAL_generated_from_LP         --score std         --cov ./COVAR_FILE.txt
1a: 2
2: 0


In [139]:
# run_prsice(GWAS_ID='ukb-b-17627')

In [140]:
!ls 'tabular_format_gwas_data/{GWAS_ID}.tsv'

tabular_format_gwas_data/ukb-b-6324.tsv


In [148]:
base_files
GWAS_ID = 'ieu-b-109'

#====

def get_prs_values(GWAS_ID):
    prsice_output = f'./prsice_output/{GWAS_ID}/'

    if False:
        with open(prsice_output+f'{GWAS_ID}.all_score', 'r') as f:
            lines = f.readlines()
            lines = ['\t'.join(x.split()) for x in lines]
            lines = '\n'.join(lines)

        with open(prsice_output+f'{GWAS_ID}.all_score.tsv', 'w') as f:
            f.write(lines)

        # best_p_val_threshold = '0.00025005'
        best_p_val_threshold = str(open(prsice_output+f'{GWAS_ID}.summary').readlines()[1].split('\t')[2]) 
    #     print(best_p_val_threshold) 
    prs_array = pd.read_csv(prsice_output+f'{GWAS_ID}.best', ' ')['PRS'].to_numpy() 
    return prs_array

In [149]:
get_prs_values(GWAS_ID=GWAS_ID).shape

(847,)

In [154]:
import numpy as np
PRS_feature_matrix = np.zeros([len(base_files), 847])
for i, GWAS_ID in enumerate(base_files):
    print(GWAS_ID)
    PRS_feature_matrix[i, :] = get_prs_values(GWAS_ID=GWAS_ID)

ieu-b-109
ukb-b-12064
ukb-b-13806
ukb-d-20405_0
ieu-b-38
ukb-b-6134
ieu-b-110
ukb-b-17627
ukb-b-19953
ukb-b-8476
ukb-d-20405_1
ukb-d-20405_2
ukb-b-2209
ukb-b-4424
ukb-b-7663
ukb-b-18275
ukb-b-770
met-d-Total_C
ieu-b-25
ieu-b-111
ukb-b-3957
ieu-b-39
ukb-b-6324


In [155]:
PRS_feature_matrix = PRS_feature_matrix.T
np.save('PRS_feature_matrix', PRS_feature_matrix)

In [207]:
PRS_feature_matrix.shape
import random

In [208]:
TRAIN_SAMPLE_SIZE = 550
train_samples = list(range(847))
random.shuffle(train_samples)
train_samples, test_samples = train_samples[:TRAIN_SAMPLE_SIZE], train_samples[TRAIN_SAMPLE_SIZE:]
print(test_samples)
print(len(train_samples), len(test_samples), len(train_samples) + len(test_samples))
PRS_feature_matrix[train_samples].mean(0), PRS_feature_matrix[train_samples].std(0)

[680, 776, 666, 297, 350, 255, 567, 492, 659, 703, 445, 433, 481, 194, 563, 505, 16, 203, 360, 269, 550, 189, 628, 89, 562, 212, 106, 416, 171, 226, 750, 134, 64, 372, 46, 441, 551, 395, 182, 248, 824, 545, 154, 66, 560, 521, 724, 797, 798, 2, 113, 15, 552, 541, 381, 274, 640, 366, 167, 769, 412, 619, 322, 749, 25, 185, 263, 169, 542, 386, 348, 312, 7, 0, 427, 44, 661, 206, 439, 457, 22, 244, 653, 556, 748, 432, 257, 501, 73, 453, 834, 469, 815, 323, 803, 305, 9, 757, 95, 643, 565, 655, 805, 158, 443, 102, 830, 440, 744, 239, 82, 631, 475, 160, 711, 423, 320, 775, 310, 608, 603, 133, 178, 705, 150, 135, 193, 165, 285, 670, 123, 245, 58, 627, 672, 318, 595, 819, 394, 5, 91, 762, 161, 327, 222, 88, 53, 101, 155, 468, 625, 8, 818, 742, 485, 687, 309, 261, 76, 414, 293, 243, 268, 376, 718, 759, 758, 115, 665, 626, 456, 681, 785, 31, 377, 497, 450, 499, 68, 408, 792, 27, 354, 514, 731, 503, 40, 615, 428, 117, 192, 172, 817, 587, 280, 786, 201, 311, 645, 37, 331, 51, 486, 437, 572, 149, 50, 

(array([-5.62869948e-05, -8.47273989e-04, -2.97113396e-02, -5.00539117e-02,
        -1.68699072e-02,  9.77735755e-03,  1.63589821e-02, -7.65790407e-03,
        -2.23720941e-02,  3.25093148e-02,  3.13638707e-02, -1.85784565e-02,
         1.07292542e-02,  5.01554176e-02,  1.15824706e-02, -4.76019952e-03,
         1.82897523e-02,  8.41139907e-03,  2.64875785e-02, -1.56245782e-03,
        -1.27972227e-02,  2.51471832e-03,  2.80220274e-03]),
 array([1.00573322, 0.96983034, 1.01992974, 0.96826368, 1.017858  ,
        1.01176271, 1.00460765, 0.98492692, 0.99783163, 1.02165108,
        0.99074247, 0.96317675, 0.98298781, 0.98236033, 1.00817736,
        1.00947234, 1.01477517, 0.97079401, 0.96252311, 0.99198965,
        1.001721  , 1.00123835, 0.98816523]))

In [199]:
PRS_feature_matrix[test_samples].mean(0), PRS_feature_matrix[test_samples].std(0)

(array([ 0.10155704,  0.03220885,  0.02282488,  0.05720392,  0.03911068,
        -0.0191249 ,  0.0223922 , -0.0174565 ,  0.06714517, -0.00862295,
        -0.01645283, -0.00248299, -0.01792296,  0.01780421,  0.02717998,
         0.0086679 ,  0.02624121, -0.0168475 , -0.03030379, -0.00472855,
        -0.04775907,  0.01003727, -0.02370823]),
 array([1.0351136 , 0.96943692, 0.98878438, 1.0215462 , 1.01929048,
        1.01102091, 0.99054645, 0.97888098, 0.9736006 , 0.89606504,
        1.02947264, 0.97359088, 1.03324595, 0.9597498 , 1.01367762,
        0.93638865, 1.05120734, 0.99516701, 1.012609  , 1.00799338,
        1.02514706, 1.02002806, 0.98244905]))

In [200]:
PRS_feature_matrix[:].mean(0), PRS_feature_matrix[:].std(0)

(array([-2.18453386e-11,  1.59386187e-12, -3.70484134e-11, -5.25786357e-11,
         4.63648188e-11, -3.73435529e-11,  4.16528834e-11, -2.54746202e-11,
        -2.55017668e-11, -1.07497054e-10, -1.12987013e-10,  3.44510085e-11,
        -8.95749433e-12,  3.71310496e-11,  1.31168894e-11,  1.97709733e-11,
        -1.55289209e-11,  8.44698930e-11,  4.45548926e-11,  2.98193646e-11,
        -8.45737962e-11, -7.35537109e-11, -4.37426928e-12]),
 array([0.99940951, 0.99940951, 0.99940951, 0.99940951, 0.99940951,
        0.99940951, 0.99940951, 0.99940951, 0.99940951, 0.99940951,
        0.99940951, 0.99940951, 0.99940951, 0.99940951, 0.99940951,
        0.99940951, 0.99940951, 0.99940951, 0.99940951, 0.99940951,
        0.99940951, 0.99940951, 0.99940951]))

In [237]:
import json
GWAS_ID = 'ieu-b-25'
usable_samples_ADNI = pd.read_csv(f'./prsice_output/{GWAS_ID}/{GWAS_ID}.best', ' ')['IID'].to_numpy().tolist()
usable_samples_ADNI = {sample:idx for idx,sample in enumerate(usable_samples_ADNI)}
json.dump(usable_samples_ADNI, open('usable_samples_ADNI.json', 'w'))
type(usable_samples_ADNI)

dict

In [212]:
!ls prsice_output

ieu-b-109  ieu-b-39	  ukb-b-13806  ukb-b-3957  ukb-b-770
ieu-b-110  lib		  ukb-b-17627  ukb-b-4424  ukb-b-8476
ieu-b-111  met-d-Total_C  ukb-b-18275  ukb-b-6134  ukb-d-20405_0
ieu-b-25   temp		  ukb-b-19953  ukb-b-6324  ukb-d-20405_1
ieu-b-38   ukb-b-12064	  ukb-b-2209   ukb-b-7663  ukb-d-20405_2


In [239]:
import json
PRS_feature_matrix = np.load('./PRS_feature_matrix.npy')
usable_samples_ADNI = json.load(open('./usable_samples_ADNI.json'))
print(PRS_feature_matrix.shape, usable_samples_ADNI.__len__())

usable_samples_ADNI

(847, 23) 847


{'014_S_0520': 0,
 '005_S_1341': 1,
 '012_S_0803': 2,
 '018_S_0055': 3,
 '027_S_0118': 4,
 '053_S_0389': 5,
 '041_S_0262': 6,
 '005_S_1224': 7,
 '137_S_0825': 8,
 '128_S_0266': 9,
 '067_S_0243': 10,
 '033_S_1116': 11,
 '007_S_0070': 12,
 '133_S_1170': 13,
 '128_S_0522': 14,
 '141_S_1245': 15,
 '007_S_1304': 16,
 '012_S_1212': 17,
 '006_S_0547': 18,
 '009_S_0751': 19,
 '123_S_0390': 20,
 '100_S_0296': 21,
 '010_S_0422': 22,
 '018_S_0080': 23,
 '018_S_0155': 24,
 '136_S_0873': 25,
 '057_S_1265': 26,
 '128_S_0608': 27,
 '116_S_1271': 28,
 '123_S_0091': 29,
 '116_S_1232': 30,
 '041_S_1435': 31,
 '021_S_0231': 32,
 '027_S_0835': 33,
 '013_S_1035': 34,
 '041_S_1418': 35,
 '141_S_1255': 36,
 '002_S_1018': 37,
 '057_S_1217': 38,
 '126_S_1077': 39,
 '114_S_0173': 40,
 '005_S_0553': 41,
 '123_S_0106': 42,
 '126_S_0606': 43,
 '022_S_0750': 44,
 '123_S_0094': 45,
 '041_S_1010': 46,
 '126_S_1340': 47,
 '011_S_0005': 48,
 '062_S_1299': 49,
 '033_S_0923': 50,
 '033_S_0511': 51,
 '032_S_0978': 52,
 '0

In [253]:
THRESHOLD_MONTH = 24

ADNI_TEMP = ADNIMERGE[['RID', 'PTID', 'DX', 'Month']].dropna().sort_values(by=['PTID', 'Month'])
Dementia = ADNI_TEMP[ADNI_TEMP['DX']=='Dementia']
Samples_Dementia = set(Dementia['PTID'].unique())
Samples_NonDementia = set(ADNI_TEMP['PTID'].unique()) - Samples_Dementia
# Samples_NonDementia
Final_Samples_Dementia = set()
for sample in Samples_Dementia:
    last_dx = ADNI_TEMP[ADNI_TEMP['PTID']==sample]['DX'].iloc[-1]
    if last_dx == 'Dementia' and sample in usable_samples_ADNI:
            Final_Samples_Dementia = Final_Samples_Dementia.union({sample})

Final_Samples_NonDementia = set()
for sample in Samples_NonDementia:
    last_month_of_dx = ADNI_TEMP[ADNI_TEMP['PTID']==sample]['Month'].iloc[-1]
    if last_month_of_dx >= THRESHOLD_MONTH and sample in usable_samples_ADNI:
        Final_Samples_NonDementia = Final_Samples_NonDementia | {sample} # union symbol in python sets
    
# sum(ADNI_TEMP['Month'] == 6), ADNI_TEMP.shape
# Dementia
Final_Samples_Dementia = [[x, 1] for x in Final_Samples_Dementia]
Final_Samples_NonDementia = [[x, 0] for x in Final_Samples_NonDementia]

Final_Samples = Final_Samples_Dementia + Final_Samples_NonDementia
json.dump(Final_Samples, open('Final_Samples.json', 'w'))

Final_Samples_Dementia.__len__(), Final_Samples_NonDementia.__len__(), Final_Samples.__len__()

(370, 398, 768)

In [261]:
ALL_GWAS_IDS = [key for key in base_files]
print(ALL_GWAS_IDS)

['ieu-b-109', 'ukb-b-12064', 'ukb-b-13806', 'ukb-d-20405_0', 'ieu-b-38', 'ukb-b-6134', 'ieu-b-110', 'ukb-b-17627', 'ukb-b-19953', 'ukb-b-8476', 'ukb-d-20405_1', 'ukb-d-20405_2', 'ukb-b-2209', 'ukb-b-4424', 'ukb-b-7663', 'ukb-b-18275', 'ukb-b-770', 'met-d-Total_C', 'ieu-b-25', 'ieu-b-111', 'ukb-b-3957', 'ieu-b-39', 'ukb-b-6324']


In [264]:
df

,GWAS_ID,Trait,Category
0,bbj-a-46,Pulse Pressure,Continuous
1,bbj-a-78,Smoking behaviors : Smoking initiation,Binary
2,ukb-b-13806,"Non-cancer illness code, self-reported: type 2...",Binary
3,ieu-a-1283,Alcohol consumption,Risk factor
4,ukb-d-20405_0,"Ever had known person concerned about, or reco...",Binary
5,ukb-d-20405_1,"Ever had known person concerned about, or reco...",Binary
6,ukb-d-20405_2,"Ever had known person concerned about, or reco...",Binary
7,met-d-Total_C,Total cholesterol,Continuous
8,ieu-b-109,HDL cholesterol,Continuous
9,ieu-b-110,LDL cholesterol,Continuous


In [273]:
df = pd.read_excel('GWAS_datasets_to_consider.xlsx', engine='openpyxl')[['GWAS_ID', 'Trait', 'Category']]
all_traits = []
for GWAS_ID in ALL_GWAS_IDS:
    trait = str(df[df['GWAS_ID'] == GWAS_ID]['Trait'].to_numpy()[0])
    print(GWAS_ID, ':', trait)
    all_traits += [trait]
print(all_traits)

ieu-b-109 : HDL cholesterol
ukb-b-12064 : Non-cancer illness code, self-reported: depression
ukb-b-13806 : Non-cancer illness code, self-reported: type 2 diabetes
ukb-d-20405_0 : Ever had known person concerned about, or recommend reduction of, alcohol consumption: No
ieu-b-38 : systolic blood pressure
ukb-b-6134 : Age completed full time education
ieu-b-110 : LDL cholesterol
ukb-b-17627 : Non-oily fish intake
ukb-b-19953 : Body mass index (BMI)
ukb-b-8476 : Loneliness, isolation
ukb-d-20405_1 : Ever had known person concerned about, or recommend reduction of, alcohol consumption: Yes, but not in the last year
ukb-d-20405_2 : Ever had known person concerned about, or recommend reduction of, alcohol consumption: Yes, during the last year
ukb-b-2209 : Oily fish intake
ukb-b-4424 : Sleep duration
ukb-b-7663 : Types of physical activity in last 4 weeks: Strenuous sports
ukb-b-18275 : Hearing difficulty/problems with background noise
ukb-b-770 : Other meat intake
met-d-Total_C : Total chole

In [275]:
import torch
torch.__version__

'1.2.0'